<a href="https://colab.research.google.com/github/Rana-Banerjee/mlpipelines/blob/main/zindi_t5_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers[torch] datasets evaluate rouge_score sacrebleu bleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Us

In [3]:
from abc import ABC, abstractmethod
import pandas as pd

class Dataset(ABC):
    _name: str
    _df: pd.DataFrame

    @property
    def name(self)->str:
        return self._name

    @name.setter
    def name(self, value)-> None:
        self._name = value

    @property
    def df(self)->pd.DataFrame:
        return self._df

    @df.setter
    def df(self, value)-> None:
        self._df = value

    @abstractmethod
    def preprocess(self)->None:
        pass

In [4]:
import pandas as pd
import ast
import os
import datasets

class Zindi_Dataset(Dataset):
    """The dyula to french translation dataset for zindi."""
    def __init__(self, data_path:str):
        self.data_path= data_path
        self.src_lang = 'dyu'
        self.target_lang = 'fr'
        self.df=None
        self.ds=None

    def load(self, seperator='|')-> None:
        """Load the csv and return the dataframe."""
        self.df= pd.read_csv(self.data_path, sep=seperator)

    def preprocess(self)-> None:
        columns = self.df.columns
        self.df[self.src_lang]=self.df['translation'].apply(lambda x: f'translate dyula to french: {ast.literal_eval(x)[self.src_lang].lower()}')
        self.df[self.target_lang]=self.df['translation'].apply(lambda x: ast.literal_eval(x)[self.target_lang].lower())
        self.df.drop(columns=columns, inplace=True)
        self.ds = datasets.Dataset.from_pandas(self.df)

In [5]:
import datasets
train_ds = Zindi_Dataset('/content/drive/MyDrive/zindi_opt/t5/data/train.csv')
train_ds.load()
train_ds.preprocess()

test_ds = Zindi_Dataset('/content/drive/MyDrive/zindi_opt/t5/data/validation.csv')
test_ds.load()
test_ds.preprocess()

train_dataset=train_ds.ds
test_dataset=test_ds.ds


In [73]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import AutoConfig

model_checkpoint= 'google-t5/t5-small'
configuration = AutoConfig.from_pretrained(model_checkpoint)
# configuration.classifier_dropout = 0.001
configuration.dropout_rate = 0.13
configuration.max_length=128

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint,config = configuration)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model.config

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


T5Config {
  "_name_or_path": "google-t5/t5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.125,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "max_length": 128,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": tru

In [74]:
def batch_tokenize_fn(examples):
    max_source_length = 128
    max_target_length = 128
    sources = examples['dyu']
    targets = examples['fr']
    model_inputs = tokenizer(sources, max_length=max_source_length, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)
        model_inputs['labels']=labels['input_ids']
    # print(model_inputs.features)
    return model_inputs

train_tokenized_ds = train_dataset.map(batch_tokenize_fn, batched=True, remove_columns=train_dataset.column_names)
test_tokenized_ds = test_dataset.map(batch_tokenize_fn, batched=True,remove_columns=test_dataset.column_names)

Map:   0%|          | 0/8065 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1471 [00:00<?, ? examples/s]

In [75]:
from transformers import DataCollatorForSeq2Seq
from dataclasses import dataclass
import torch
import numpy as np
import random
import os

@dataclass
class Config:

    batch_size: int = 32
    num_workers: int = 4
    seed: int = 42
    max_source_length: int = 128
    max_target_length: int = 128

    lr: float = 0.0001
    weight_decay: float = 0.01
    epochs: int = 5000
    device: torch.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def __post_init__(self):
        random.seed(self.seed)
        np.random.seed(self.seed)
        torch.manual_seed(self.seed)
        torch.cuda.manual_seed_all(self.seed)

config = Config()
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)



In [76]:
from transformers import Seq2SeqTrainingArguments
args = Seq2SeqTrainingArguments(
    output_dir='/content/drive/MyDrive/zindi_opt/t5/model',
    evaluation_strategy="steps",
    learning_rate=config.lr,
    per_device_train_batch_size=config.batch_size,
    per_device_eval_batch_size=config.batch_size,
    weight_decay=config.weight_decay,
    save_total_limit=2,
    num_train_epochs=config.epochs,
    predict_with_generate=True,
    load_best_model_at_end=True,
    greater_is_better=True,
    metric_for_best_model="rougeL",
    gradient_accumulation_steps=8,
    do_train=True,
    # careful when attempting to train t5 models on fp16 mixed precision,
    # the model was trained on bfloat16 mixed precision, and mixing different mixed precision
    # type might result in nan loss
    # https://discuss.huggingface.co/t/mixed-precision-for-bfloat16-pretrained-models/5315
    fp16=True,
    dataloader_num_workers=2,
    # logging_steps=500
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [77]:
import evaluate
rouge_score = evaluate.load("rouge")
bleu_score = evaluate.load("bleu")
sacrebleu_score = evaluate.load("sacrebleu")

def compute_metrics(eval_pred):
    """
    Compute rouge and bleu metrics for seq2seq model generated prediction.

    tip: we can run trainer.predict on our eval/test dataset to see what a sample
    eval_pred object would look like when implementing custom compute metrics function
    """
    predictions, labels = eval_pred
    # Decode generated summaries, which is in ids into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode labels, a.k.a. reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge_score.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        rouge_types=["rouge1", "rouge2", "rougeL"]
    )
    score = sacrebleu_score.compute(
        predictions=decoded_preds,
        references=decoded_labels
    )
    result["sacrebleu"] = score["score"]
    # bscore=bleu_score.compute(
    #     predictions=decoded_preds,
    #     references=decoded_labels
    # )
    # result["bleu"] = bscore["bleu"]
    return {k: round(v, 4) for k, v in result.items()}

In [78]:
from transformers import Seq2SeqTrainer,EarlyStoppingCallback
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_tokenized_ds,
    eval_dataset=test_tokenized_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback()]
)

In [ ]:
train_output = trainer.train()

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Sacrebleu
500,2.992200,2.574626,0.118600,0.019200,0.113900,0.966300
1000,2.603500,2.467020,0.142100,0.031000,0.136300,1.484700
1500,2.383900,2.415823,0.157300,0.040300,0.151200,2.269400
2000,2.211800,2.388616,0.176400,0.052700,0.170900,2.783200
2500,2.061700,2.373447,0.189300,0.062200,0.183600,3.672300
3000,1.930500,2.377022,0.191700,0.064600,0.186300,3.943900


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1283: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation paramet